In [1]:
import sys
import os
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import tqdm
from tqdm.notebook import tqdm
import re
import urllib.request

In [4]:
data = pd.read_excel('moviedata.xlsx')

In [5]:
list(data['영화명'])[0]

'카오산 탱고'

In [2]:
#query_txt = list(data['영화명'])
start_date="20100101"
end_date="20200718"
#q = '"영화"'

In [35]:
path = 'chromedriver.exe'
# options = webdriver.ChromeOptions()
# options.add_argument('headless')    # 웹 브라우저를 띄우지 않는 headless chrome 옵션 적용
# options.add_argument('disable-gpu')    # GPU 사용 안함
# options.add_argument('lang=ko_KR')    # 언어 설정, options=options
# driver = webdriver.Chrome('chromedriver', options=options)



driver = webdriver.Chrome(path)#, options=options)
driver.get('http://www.naver.com')
time.sleep(2)
# 네이버 검색창에 검색어 검색
element = driver.find_element_by_id('query')
element.send_keys("영화 ", list(data['영화명'])[0])  # query_txt는 위에서 입력한 '영화 리뷰'
element.submit()

# 블로그 카테고리 선택
driver.find_element_by_link_text('블로그').click()

# 검색옵션 버튼 클릭

driver.find_element_by_id('_search_option_btn').click()
    
#날짜 입력
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[2]/a""").click()
time.sleep(2)

#날짜 입력
s_date = driver.find_element_by_xpath("""//*[@id="blog_input_period_begin"]""")     
driver.find_element_by_xpath("""//*[@id="blog_input_period_begin"]""").click()
s_date.clear()  # 날짜 입력 부분에 기존에 입력되어 있던 날짜를 제거합니다. 
time.sleep(1)
    
# 아래 코드가 날짜를 for 반복문으로 1 글자씩 입력
for c in start_date:
    s_date.send_keys(c)
    time.sleep(0.1)

# 종료 날짜 입력
e_date = driver.find_element_by_xpath("""//*[@id="blog_input_period_end"]""")
driver.find_element_by_xpath("""//*[@id="blog_input_period_end"]""").click()
e_date.clear()
time.sleep(1)

for c in end_date:
    e_date.send_keys(c)
    time.sleep(0.1)

driver.find_element_by_class_name('tx').click()
time.sleep(3)
    
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]""").click()
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/div/ul/li[1]/a""").click()
time.sleep(1)
    
    
url_list = []
title_list = []

# 몇개의 페이지를 크롤링 할지 선택

total_page = 1
for j in tqdm(list(data['영화명'])[:3]) : 
    for i in tqdm(range(total_page)) : # 페이지 번호
        i = i * 1 
        url = "https://search.naver.com/search.naver?date_from={0}&date_option=8&date_to={1}&dup_remove=1&nso=p%3Afrom{2}to{3}&post_blogurl=&post_blogurl_without=&query=영화%20{4}&sm=tab_pge&srchby=all&st=sim&where=post&start={5}".format(start_date,end_date,start_date,end_date, j, i)
   
        driver.get(url)
        time.sleep(0.5)
        
 # URL 크롤링 시작
    titles = "a.sh_blog_title._sp_each_url._sp_each_title"
    moive_raw = driver.find_elements_by_css_selector(titles)

    # url 크롤링 시작    
    for movie in moive_raw:
        url = movie.get_attribute('href')   
        url_list.append(url)
    
     # 제목   
for i in list(data['영화명']) :
    for j in range(1, 11) :
        title_list.append(i)

    
print('url갯수: ', len(url_list))
print('title갯수: ', len(title_list))



df = pd.DataFrame({'url':url_list, 'title':title_list[:30]})

# 저장하기
df.to_excel("blog_url.xlsx")


url_load = pd.read_excel("blog_url.xlsx", index_col=0)        # 기본 모델
num_list = len(url_load)
    

dict = {}  # 전체 크롤링 데이터 저장

# ★수집할 글 갯수
#number = url_list
for i in tqdm(range(0, num_list)) : 
    # 글 띄우기
    url = url_load['url'][i]
    driver = webdriver.Chrome("chromedriver.exe")
    driver.get(url)   # 글 띄우기
    
    # 크롤링
    
    try : 
        # iframe 접근
        driver.switch_to_frame('mainFrame')

        target_info = {}

        # 제목 크롤링 시작
        overlays = ".se-fs-.se-ff-"                                 
        tit = driver.find_element_by_css_selector(overlays)         # title
        title = tit.text
        title
        
        # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"                                 
        contents = driver.find_elements_by_css_selector(overlays)         # date

        content_list = []
        for content in contents:
            content_list.append(content.text)

        content_str = ' '.join(content_list)

        # 글 하나는 target_info
        target_info['title'] = title
        target_info['content'] = content_str

        # 각각의 글은 dict라는 딕셔너리에.
        dict[i] = target_info
        time.sleep(1)
        
        print(i, title)

        # 글 하나 크롤링 후 크롬 창 닫기
        driver.close()       
    
    # 에러나면 현재 크롬창 닫고 다음 글(i+1)로 이동
    except:
        driver.close()
        time.sleep(1)
        continue
    
    # 중간,중간에 파일로 저장하기
#     if i == 30 or 50 or 80:
#         # 판다스로 만들기
#         import pandas as pd
#         result_df = pd.DataFrame.from_dict(dict, 'index')

#         # 저장하기
#         result_df.to_excel("blog_content_movie.xlsx")
#         time.sleep(3)

print('수집한 글 갯수: ', len(dict))
print(dict)

# 판다스로 만들기
import pandas as pd
result_df = pd.DataFrame.from_dict(dict, 'index')
result_df.to_excel("blog_content_movie.xlsx")

df2 = pd.read_excel('blog_content_movie.xlsx', index_col=0)
a = pd.DataFrame(title_list)
df2['title2'] = a[:30]
df2.to_csv('blog_content_movie2.csv', encoding='utf-8-sig')



url갯수:  30
title갯수:  82380


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:116: DeprecationWarning: use driver.switch_to.frame instead


1 홍완표-현리-오창경 & 김범삼 감독의 태국 낭만 여행기 <카오산 탱고>, 2020.07 개봉 확정!
2 
3 7월 여름, 본격 스크린 태국 여행!
4 카오산 탱고,2020
10 영화 욕창) 그 속을 어찌 알까...
11 "누구나" - 심혜정 감독의 영화 "욕창"을 보고
12 [영화를 보았다] 욕창 (A Bedsore), 2019
13 직업적 해이로 시작된 병에서 도덕적 해이로 스스로의 권위를 무너뜨리다. 영화 <욕창> 리뷰
14 [영화] 욕창 시사회 및 스포 없는 후기 _ 스포 있는 후기는 다음 주에?
15 영화 욕창 포스터
18 심혜정 감독 & 김종구-김애심-전국향 배우의 브졸국제아시아영화제 2관왕 수상작 <욕창>, 2020.07 개봉 확정!
19 <욕창> 겉으로 봐서는 모르는 욕창의 냄새
20 [공연] 팬텀싱어3, 부산 네오필하모닉오케스트라 정기공연, 독립영화 바다로 가자.
21 지극히 사적이고 가장 보편적인 한반도의 트라우마를 만난다.
22 영화 『바다로 가자』 시네토크(영상포함)
24 [영화] 바다로 가자
25 
26 신작 영화 바다로 가자 시사회 이벤트가 진행중입니다
27 
28 영화 바다로 가자, 실향민들의 끝나지 않은 이야기
29 김량 감독의 한반도 분단 트라우마에 관한 딥포커스 다큐멘터리 <바다로 가자>, 2020.06.18 개봉!

수집한 글 갯수:  21
{1: {'title': '홍완표-현리-오창경 & 김범삼 감독의 태국 낭만 여행기 <카오산 탱고>, 2020.07 개봉 확정!', 'content': '외출은 물론 해외여행까지도 어려움을 겪고 있는 이 시국, 방구석 방콕러를 자처한 많은 관객들에게 생생한 스크린 여행을 선사할 영화 <카오산 탱고>가 이국적인 감성을 가득 담은 메인 포스터, 메인 예고편 영상, 보도스틸 10종을 공개했다.  \n<카오산 탱고>는 시나리오 취재를 위해 태국 방콕에 처음 방문한 영화감독 지망생 ‘지하’(홍완표)가 프로 태국 여행러 ‘하영’(현리)과 우연히 동행하게 되면서 벌어지는 예측불허 낭만 여행기.  공개된

# 리뷰 크롤링

In [36]:
df2

,title,content,title2
1,"홍완표-현리-오창경 & 김범삼 감독의 태국 낭만 여행기 <카오산 탱고>, 2020....","외출은 물론 해외여행까지도 어려움을 겪고 있는 이 시국, 방구석 방콕러를 자처한 많...",카오산 탱고
2,NaN,<카오산 탱고>는 태국에서 만난 한국 남녀의 낭만적 여행을 그린 영화. 낭만이 결국...,카오산 탱고
3,"7월 여름, 본격 스크린 태국 여행!","7월 여름, 본격 스크린 태국 여행!\n낯선 여행지에서 두 남녀의 동행이 시작된다!...",카오산 탱고
4,"카오산 탱고,2020",갑작스레 코로나가 창궐하여 끝을 알 수 없는 코비드19 시대가.\n계속되고 있다.\...,카오산 탱고
10,영화 욕창) 그 속을 어찌 알까...,"욕창\nA Bedsore, 2019 겉만 보고는 알 수 없는 깊이와 너비. 러닝타임...",욕창
11,"""누구나"" - 심혜정 감독의 영화 ""욕창""을 보고",욕창은 환자를 제대로 돌보지 못했다는 상징과 같은 질병이다. 욕창은 단기간에 생기지...,욕창
12,"[영화를 보았다] 욕창 (A Bedsore), 2019",200629 오오극장 관객프로그래머 프리뷰로 본 영화. \n결국 희망이라는 것은 저...,욕창
13,직업적 해이로 시작된 병에서 도덕적 해이로 스스로의 권위를 무너뜨리다. 영화 <욕창...,※ 포스팅에 앞서 스포일러가 포함될 수 있음을 알립니다. ※ 퇴직 공무원 창식은 간...,욕창
14,[영화] 욕창 시사회 및 스포 없는 후기 _ 스포 있는 후기는 다음 주에?,안녕하세요 시우입니다. 정말이지 오랜만에 영화라는 걸 영화관에서 보고 왔네요. 평소...,욕창
15,영화 욕창 포스터,"2018년도에도 영화를 백 편이상 보았고 (독립, 상업 할 것 없이 장편만!)\n올...",욕창


In [30]:
movie_raw2 = []

In [31]:
for i in list(data['영화명']) :
    for j in range(1, 11) :
        moive_raw2.append(i)

In [32]:
a = pd.DataFrame(moive_raw2)

In [33]:
a[:10]

,0
0,카오산 탱고
1,카오산 탱고
2,카오산 탱고
3,카오산 탱고
4,카오산 탱고
5,카오산 탱고
6,카오산 탱고
7,카오산 탱고
8,카오산 탱고
9,카오산 탱고
